In [3]:
# Imports inside model.py
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import copy
import astropy.units as u
import operator
from gammapy.utils.fitting import Parameters, Parameter
from gammapy.utils.scripts import make_path
from gammapy.maps import Map, MapAxis


In [4]:
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord, Angle
from gammapy.irf import (
    EffectiveAreaTable2D,
    EnergyDispersion2D,
    EnergyDependentMultiGaussPSF,
    Background3D,
)
from gammapy.maps import WcsGeom, MapAxis, WcsNDMap, Map
from gammapy.spectrum.models import PowerLaw
from gammapy.image.models import SkyGaussian
from gammapy.cube.models import SkyModel, SkyModels
from gammapy.cube import MapFit, MapEvaluator, PSFKernel
from gammapy.cube import make_map_exposure_true_energy, make_map_background_irf

In [5]:
import os
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord
from gammapy.extern.pathlib import Path
from gammapy.data import DataStore
from gammapy.irf import EnergyDispersion, make_mean_psf, make_mean_edisp
from gammapy.maps import WcsGeom, MapAxis, Map, WcsNDMap
from gammapy.cube import MapMaker, MapEvaluator, PSFKernel, MapFit
from gammapy.cube.models import SkyModel, SkyDiffuseCube
from gammapy.spectrum.models import PowerLaw, ExponentialCutoffPowerLaw
from gammapy.image.models import SkyGaussian, SkyPointSource
from regions import CircleSkyRegion

### Class for background model

In [6]:
from gammapy.cube.models import SkyModelBase

In [7]:
class BackgroundCube(SkyModelBase):
    
    def __init__(self, map, norm=1, meta=None):
        #axis = map.geom.get_axis_by_name("energy")
        #if axis.node_type != "edges":
        #    raise ValueError('Need a map with energy axis node_type="edges"')

        self.map = map
        self.meta = {} if meta is None else meta
        self.parameters = Parameters([Parameter("norm", norm)])
    
    def evaluate(self):
        #TODO : Add energy dependance - pass an index and scale each slice accordingly
        return u.Quantity(self.parameters["norm"].value * self.map.data, self.map.unit, copy=False)
        

Creating background model - seems to work - but should the class return a map or just an array?

In [10]:
axis = MapAxis.from_edges([0.1, 1.0, 10.0, 100], name="energy", unit="TeV", interp="log")
m = Map.create(npix=(4, 3), binsz=2, axes=[axis], unit="cm-2 s-1 MeV-1 sr-1")
m.data += 1.0
bkg = BackgroundCube(m, norm=2)

In [13]:
back_model = bkg.evaluate()
back_model

<Quantity [[[2., 2., 2., 2.],
            [2., 2., 2., 2.],
            [2., 2., 2., 2.]],

           [[2., 2., 2., 2.],
            [2., 2., 2., 2.],
            [2., 2., 2., 2.]],

           [[2., 2., 2., 2.],
            [2., 2., 2., 2.],
            [2., 2., 2., 2.]]] 1 / (cm2 MeV s sr)>

Now MapFit and Evaluator. 

In [45]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
from astropy.utils import lazyproperty
import astropy.units as u
from gammapy.utils.fitting import Fit
from gammapy.stats import cash
from gammapy.maps import Map, MapAxis

class MapEvaluator1(object):
   

    def __init__(
        self, model=None, exposure=None, background=None, psf=None, edisp=None
    ):
        self.model = model
        self.exposure = exposure
        self.background = background
        self.psf = psf
        self.edisp = edisp

    @lazyproperty
    def geom(self):
        """This will give the energy axes in e_true"""
        return self.exposure.geom

    @lazyproperty
    def geom_image(self):
        return self.geom.to_image()

    @lazyproperty
    def energy_center(self):
        """True energy axis bin centers (`~astropy.units.Quantity`)"""
        energy_axis = self.geom.get_axis_by_name("energy")
        energy = energy_axis.center * energy_axis.unit
        return energy[:, np.newaxis, np.newaxis]

    @lazyproperty
    def energy_edges(self):
        """Energy axis bin edges (`~astropy.units.Quantity`)"""
        energy_axis = self.geom.get_axis_by_name("energy")
        energy = energy_axis.edges * energy_axis.unit
        return energy[:, np.newaxis, np.newaxis]

    @lazyproperty
    def energy_bin_width(self):
        """Energy axis bin widths (`astropy.units.Quantity`)"""
        return np.diff(self.energy_edges, axis=0)

    @lazyproperty
    def lon_lat(self):
        """Spatial coordinate pixel centers.

        Returns ``lon, lat`` tuple of `~astropy.units.Quantity`.
        """
        lon, lat = self.geom_image.get_coord()
        return lon * u.deg, lat * u.deg

    @lazyproperty
    def lon(self):
        return self.lon_lat[0]

    @lazyproperty
    def lat(self):
        return self.lon_lat[1]

    @lazyproperty
    def solid_angle(self):
        """Solid angle per pixel"""
        return self.geom.solid_angle()

    @lazyproperty
    def bin_volume(self):
        """Map pixel bin volume (solid angle times energy bin width)."""
        omega = self.solid_angle
        de = self.energy_bin_width
        return omega * de

    def compute_dnde(self):
        """Compute model differential flux at map pixel centers.

        Returns
        -------
        model_map : `~gammapy.maps.Map`
            Sky cube with data filled with evaluated model values.
            Units: ``cm-2 s-1 TeV-1 deg-2``
        """
        coord = (self.lon, self.lat, self.energy_center)
        dnde = self.model.evaluate(*coord)
        return dnde

    def compute_flux(self):
        """Compute model integral flux over map pixel volumes.

        For now, we simply multiply dnde with bin volume.
        """
        dnde = self.compute_dnde()
        volume = self.bin_volume
        flux = dnde * volume
        return flux.to("cm-2 s-1")

    def apply_exposure(self, flux):
        """Compute npred cube

        For now just divide flux cube by exposure
        """
        npred = Map.from_geom(self.geom, unit="")
        npred.data = (flux * self.exposure.quantity).to("").value
        return npred

    def apply_psf(self, npred):
        """Convolve npred cube with PSF"""
        return npred.convolve(self.psf)

    def apply_edisp(self, npred):
        """Convolve map data with energy dispersion.

        Parameters
        ----------
        npred : `~gammapy.maps.Map`
            Predicted counts in true energy bins

        Returns
        ---------
        npred_reco : `~gammapy.maps.Map`
            Predicted counts in reco energy bins
        """
        loc = npred.geom.get_axis_index_by_name("energy")
        data = np.rollaxis(npred.data, loc, len(npred.data.shape))
        data = np.dot(data, self.edisp.pdf_matrix)
        data = np.rollaxis(data, -1, loc)
        e_reco_axis = MapAxis.from_edges(
            self.edisp.e_reco.bins, unit=self.edisp.e_reco.unit
        )
        geom_ereco = self.exposure.geom.to_image().to_cube(axes=[e_reco_axis])
        npred = Map.from_geom(geom_ereco, unit="")
        npred.data = data
        return npred
    
    def apply_bkg(self, npred):
        #background is a model here
        bkg = self.background.evaluate()
        npred.data = npred.data + bkg.value #check units and add properly
        return npred
        

    def compute_npred(self):
        """
        Evaluate model predicted counts.

        Returns
        -------
        npred.data : ~numpy.ndarray
            array of the predicted counts in each bin (in reco energy)
        """
        flux = self.compute_flux()
        npred = self.apply_exposure(flux)
        if self.psf is not None:
            npred = self.apply_psf(npred)
        if self.edisp is not None:
            npred = self.apply_edisp(npred)
        #test if self.background is a model. else, do as before
        npred = self.apply_bkg(npred)
            
        #if self.background:
        #    npred.data += self.background.data
        return npred.data


In [46]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord, Angle
from gammapy.irf import (
    EffectiveAreaTable2D,
    EnergyDispersion2D,
    EnergyDependentMultiGaussPSF,
    Background3D,
)
from gammapy.maps import WcsGeom, MapAxis, WcsNDMap, Map
from gammapy.spectrum.models import PowerLaw
from gammapy.image.models import SkyGaussian
from gammapy.cube.models import SkyModel, SkyModels
from gammapy.cube import MapFit, MapEvaluator, PSFKernel
from gammapy.cube import make_map_exposure_true_energy, make_map_background_irf

In [47]:
def get_irfs():
    """Load CTA IRFs"""
    filename = "$GAMMAPY_DATA/cta-1dc/caldb/data/cta/1dc/bcf/South_z20_50h/irf_file.fits"
    psf = EnergyDependentMultiGaussPSF.read(
        filename, hdu="POINT SPREAD FUNCTION"
    )
    aeff = EffectiveAreaTable2D.read(filename, hdu="EFFECTIVE AREA")
    edisp = EnergyDispersion2D.read(filename, hdu="ENERGY DISPERSION")
    bkg = Background3D.read(filename, hdu="BACKGROUND")
    return dict(psf=psf, aeff=aeff, edisp=edisp, bkg=bkg)


irfs = get_irfs()

In [48]:
# Define sky model to simulate the data
spatial_model = SkyGaussian(lon_0="0.2 deg", lat_0="0.1 deg", sigma="0.3 deg")
spectral_model = PowerLaw(
    index=3, amplitude="1e-11 cm-2 s-1 TeV-1", reference="1 TeV"
)
sky_model = SkyModel(
    spatial_model=spatial_model, spectral_model=spectral_model
)


In [49]:
# Define map geometry
axis = MapAxis.from_edges(
    np.logspace(-1., 1., 10), unit="TeV", name="energy", interp="log"
)
geom = WcsGeom.create(
    skydir=(0, 0), binsz=0.02, width=(5, 4), coordsys="GAL", axes=[axis]
)
pointing = SkyCoord(1, 0.5, unit="deg", frame="galactic")
livetime = 1 * u.hour
offset_max = 2 * u.deg
offset = Angle("2 deg")
exposure = make_map_exposure_true_energy(
    pointing=pointing, livetime=livetime, aeff=irfs["aeff"], geom=geom
)

In [72]:
back_map = Map.from_geom(geom=geom)
back_map += 1.0
background = BackgroundCube(back_map, norm = 3.0)
b1 = background.evaluate()


In [73]:
evaluator = MapEvaluator1(
    model=sky_model, exposure=exposure, background=background
)

In [74]:
npred = evaluator.compute_npred()

In [75]:
npred_map = WcsNDMap(geom, npred)
rng = np.random.RandomState(seed=42)
counts = rng.poisson(npred)
counts_map = WcsNDMap(geom, counts)

model creating works

Now for the fitting

In [ ]:
# Define sky model to fit the data
spatial_model = SkyGaussian(lon_0="0 deg", lat_0="0 deg", sigma="1 deg")
spectral_model = PowerLaw(
    index=2, amplitude="1e-11 cm-2 s-1 TeV-1", reference="1 TeV"
)
model = SkyModel(spatial_model=spatial_model, spectral_model=spectral_model)

# Impose that specrtal index remains within limits
spectral_model.parameters['index'].min = 0.
spectral_model.parameters['index'].max = 10.


In [ ]:
class MapFit1(Fit):
   
    def __init__(
        self, model, counts, exposure, background=None, mask=None, psf=None, edisp=None
    ):
        if mask is not None and mask.data.dtype != np.dtype("bool"):
            raise ValueError("mask data must have dtype bool")

        # Create a copy of the input model that is owned and modified by MapFit
        self._model = model.copy()
        self.counts = counts
        self.exposure = exposure
        self.background = BackgroundCube(back_map, norm = 3.0)
        self.mask = mask
        self.psf = psf
        self.edisp = edisp

        self.evaluator = MapEvaluator(
            model=self._model,
            exposure=exposure,
            background=self.background,
            psf=self.psf,
            edisp=self.edisp,
        )

    @property
    def stat(self):
        """Likelihood per bin given the current model parameters"""
        npred = self.evaluator.compute_npred()
        return cash(n_on=self.counts.data, mu_on=npred)

    def total_stat(self, parameters):
        """Total likelihood given the current model parameters"""
        self._model.parameters = parameters
        if self.mask:
            stat = self.stat[self.mask.data]
        else:
            stat = self.stat
        return np.sum(stat, dtype=np.float64)